# 🚀 CyberPuppy 霸凌偵測訓練 - A100 優化版

**GPU**: A100 (40GB)  
**目標**: F1 ≥ 0.75  
**優化**: 大 batch + bf16 + 快速訓練  

**預計時間**: 1-2 小時（A100 加速）

---

## 1️⃣ GPU 驗證 - 確認 A100

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.1f} GB")
    
    compute_capability = torch.cuda.get_device_capability(0)
    print(f"Compute Capability: {compute_capability}")
    print(f"BF16 Support: {compute_capability[0] >= 8}")
    
    if "A100" not in gpu_name:
        print(f"\n⚠️ 警告: 當前 GPU 是 {gpu_name}，不是 A100")
        print("此 notebook 針對 A100 優化，其他 GPU 可能需要調整 batch size")
else:
    print("❌ 未檢測到 GPU！")

## 2️⃣ GitHub 認證設置

In [ ]:
from getpass import getpass

# 從 Colab Secrets 或手動輸入
try:
    from google.colab import userdata
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✅ 從 Colab Secrets 載入 token")
except:
    GITHUB_TOKEN = getpass("GitHub Token: ")

GITHUB_USERNAME = "thc1006"
REPO_NAME = "cyberbully-zh-moderation-bot"

!git config --global user.email "colab-a100@example.com"
!git config --global user.name "Colab A100 Training"

print("✅ Git 配置完成")

## 3️⃣ Clone Repository + 拉取 LFS 資料

In [ ]:
import os
import subprocess
import shutil

# 清理舊目錄
if os.path.exists(REPO_NAME):
    shutil.rmtree(REPO_NAME, ignore_errors=True)

os.chdir("/content")

# Clone
print("📥 Cloning repository...")
repo_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

os.chdir(REPO_NAME)
print(f"📁 目錄: {os.getcwd()}")

# 拉取 LFS 資料
print("\n📦 拉取 Git LFS 資料...")
!git lfs install
!git lfs pull

# 驗證訓練資料
train_file = "data/processed/training_dataset/train.json"
file_size = os.path.getsize(train_file)
print(f"\n{'✅' if file_size > 1000000 else '❌'} 訓練資料: {file_size / 1024 / 1024:.1f} MB")

## 4️⃣ 安裝依賴 - CUDA 12.x + PyTorch 2.8

In [ ]:
print("📦 安裝套件（A100 優化版本）...")

# PyTorch 2.8 + CUDA 12.6
!pip install -q --index-url https://download.pytorch.org/whl/cu126 \
  torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0

# NumPy 2.x
!pip install -q "numpy>=2,<2.3"

# Transformers + Accelerate
!pip install -q transformers==4.46.3 accelerate==1.2.1 datasets==3.2.0
!pip install -q scikit-learn==1.6.1 tqdm==4.67.1 pandas==2.2.2

# TensorBoard
!pip install -q "tensorboard~=2.19.0"

# 驗證安裝
import torch
import numpy as np
print(f"\n✅ PyTorch: {torch.__version__}")
print(f"✅ NumPy: {np.__version__}")
print(f"✅ CUDA: {torch.version.cuda}")
print(f"✅ BF16 可用: {torch.cuda.is_bf16_supported()}")

## 5️⃣ 驗證資料集

In [ ]:
import json

data_dir = "data/processed/training_dataset"

for split in ["train", "dev", "test"]:
    filepath = os.path.join(data_dir, f"{split}.json")
    file_size = os.path.getsize(filepath)
    
    if file_size > 100000:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ {split}.json: {len(data):,} 樣本 ({file_size / 1024 / 1024:.1f} MB)")
    else:
        print(f"❌ {split}.json 是 LFS pointer ({file_size} bytes)")

print("\n✅ 資料集就緒！")

## 6️⃣ A100 優化訓練配置

In [ ]:
from dataclasses import dataclass

@dataclass
class A100Config:
    name: str
    base_model: str
    learning_rate: float
    batch_size: int
    num_epochs: int
    early_stopping_patience: int
    bullying_weight: float
    accumulation_steps: int = 1

# A100 優化配置：更大 batch + 更快訓練
configs = [
    A100Config(
        name="macbert_a100_optimized",
        base_model="hfl/chinese-macbert-base",
        learning_rate=2e-5,
        batch_size=32,
        num_epochs=15,
        early_stopping_patience=4,
        bullying_weight=2.5,
        accumulation_steps=2,  # 等效 batch=64
    ),
    A100Config(
        name="roberta_a100_aggressive",
        base_model="hfl/chinese-roberta-wwm-ext",
        learning_rate=3e-5,
        batch_size=24,
        num_epochs=18,
        early_stopping_patience=3,
        bullying_weight=2.8,
        accumulation_steps=3,  # 等效 batch=72
    ),
]

print("📋 A100 訓練配置:")
for i, cfg in enumerate(configs, 1):
    effective_batch = cfg.batch_size * cfg.accumulation_steps
    print(f"{i}. {cfg.name}")
    print(f"   - Batch: {cfg.batch_size} × {cfg.accumulation_steps} = {effective_batch}")
    print(f"   - LR: {cfg.learning_rate}")
    print(f"   - BF16: Yes")

## 7️⃣ 訓練 Model A - MacBERT (A100 優化)

In [ ]:
import os
os.chdir(f"/content/{REPO_NAME}")

config = configs[0]
print(f"🚀 訓練: {config.name}")
print(f"💾 等效 Batch Size: {config.batch_size * config.accumulation_steps}")
print("="*60)

!python scripts/train_simple_with_args.py \
  --model_name {config.base_model} \
  --output_dir models/experiments/{config.name} \
  --train_file data/processed/training_dataset/train.json \
  --dev_file data/processed/training_dataset/dev.json \
  --test_file data/processed/training_dataset/test.json \
  --learning_rate {config.learning_rate} \
  --batch_size {config.batch_size} \
  --num_epochs {config.num_epochs} \
  --early_stopping_patience {config.early_stopping_patience} \
  --bullying_weight {config.bullying_weight} \
  --accumulation_steps {config.accumulation_steps} \
  --bf16

print(f"\n✅ {config.name} 訓練完成")

## 8️⃣ 訓練 Model B - RoBERTa (激進配置)

In [ ]:
config = configs[1]
print(f"🚀 訓練: {config.name}")
print(f"💾 等效 Batch Size: {config.batch_size * config.accumulation_steps}")
print("="*60)

!python scripts/train_simple_with_args.py \
  --model_name {config.base_model} \
  --output_dir models/experiments/{config.name} \
  --train_file data/processed/training_dataset/train.json \
  --dev_file data/processed/training_dataset/dev.json \
  --test_file data/processed/training_dataset/test.json \
  --learning_rate {config.learning_rate} \
  --batch_size {config.batch_size} \
  --num_epochs {config.num_epochs} \
  --early_stopping_patience {config.early_stopping_patience} \
  --bullying_weight {config.bullying_weight} \
  --accumulation_steps {config.accumulation_steps} \
  --bf16

print(f"\n✅ {config.name} 訓練完成")

## 9️⃣ 評估所有模型

In [ ]:
import json
import os
import glob

results = []

print("🔍 評估訓練結果...\n")

for config in configs:
    model_dir = f"models/experiments/{config.name}"
    
    print(f"檢查: {config.name}")
    print(f"  路徑: {model_dir}")
    
    if not os.path.exists(model_dir):
        print(f"  ❌ 目錄不存在\n")
        continue
    
    # 列出目錄內所有檔案
    files_in_dir = os.listdir(model_dir)
    print(f"  📁 目錄內容: {files_in_dir}")
    
    # 尋找評估結果
    eval_file = os.path.join(model_dir, "eval_results.json")
    final_file = os.path.join(model_dir, "final_results.json")
    
    print(f"  尋找: eval_results.json - {'✅存在' if os.path.exists(eval_file) else '❌不存在'}")
    print(f"  尋找: final_results.json - {'✅存在' if os.path.exists(final_file) else '❌不存在'}")
    
    metrics = None
    if os.path.exists(final_file):
        with open(final_file, 'r') as f:
            metrics = json.load(f)
        f1_score = metrics.get('test_bullying_f1', 0.0)
        print(f"  ✅ 使用 final_results.json")
    elif os.path.exists(eval_file):
        with open(eval_file, 'r') as f:
            metrics = json.load(f)
        f1_score = metrics.get('bullying_f1', 0.0)
        print(f"  ✅ 使用 eval_results.json")
    else:
        print(f"  ⚠️ 未找到結果檔案 - 訓練可能失敗或未完成\n")
        continue
    
    results.append({
        'name': config.name,
        'f1': f1_score,
        'path': model_dir,
        'metrics': metrics
    })
    
    status = "✅" if f1_score >= 0.75 else "⚠️"
    print(f"  {status} F1 = {f1_score:.4f}\n")

# 選出最佳模型
print("="*60)
if results:
    best_model = max(results, key=lambda x: x['f1'])
    print(f"🏆 最佳: {best_model['name']}")
    print(f"📊 F1: {best_model['f1']:.4f}")
    print("="*60)
    
    with open('best_model_a100.json', 'w') as f:
        json.dump(best_model, f, indent=2, ensure_ascii=False)
else:
    print("❌ 沒有可用結果")
    print("可能原因:")
    print("  1. 訓練腳本執行失敗（檢查上面的輸出）")
    print("  2. 發生 CUDA OOM 或其他錯誤")
    print("  3. 資料載入失敗")
    print("  4. 請向上捲動查看訓練 cell 的完整輸出")
    print("="*60)

## 🔟 複製最佳模型

In [ ]:
import shutil
import pandas as pd

if 'best_model' in locals() and best_model['f1'] >= 0.70:
    deploy_dir = "models/bullying_a100_best"
    os.makedirs(deploy_dir, exist_ok=True)

    # 驗證模型權重檔案是否存在
    model_files = [f for f in os.listdir(best_model['path'])
                   if f.endswith(('.safetensors', '.bin', '.pt', '.pth'))]

    if not model_files:
        print("❌ 錯誤: 未找到模型權重檔案！")
        print(f"檢查目錄: {best_model['path']}")
        print(f"目錄內容: {os.listdir(best_model['path'])}")
        print("\n可能原因:")
        print("1. 訓練腳本沒有正確保存模型")
        print("2. 模型保存路徑不正確")
        print("3. 訓練過程中出現錯誤")
    else:
        print(f"✅ 找到 {len(model_files)} 個模型檔案:")
        total_size = 0
        for f in model_files:
            size = os.path.getsize(os.path.join(best_model['path'], f))
            total_size += size
            print(f"  - {f}: {size / 1024 / 1024:.1f} MB")
        print(f"總大小: {total_size / 1024 / 1024:.1f} MB")

        # 複製所有檔案
        print(f"\n📦 複製到 {deploy_dir}...")
        for file in os.listdir(best_model['path']):
            src = os.path.join(best_model['path'], file)
            if os.path.isfile(src):
                shutil.copy2(src, os.path.join(deploy_dir, file))

        # 保存部署資訊
        deploy_info = {
            'model_name': best_model['name'],
            'f1_score': best_model['f1'],
            'metrics': best_model['metrics'],
            'trained_on': 'Google Colab A100',
            'timestamp': pd.Timestamp.now().isoformat(),
            'model_files': model_files,
            'total_size_mb': round(total_size / 1024 / 1024, 2)
        }

        with open(os.path.join(deploy_dir, 'deployment_info.json'), 'w') as f:
            json.dump(deploy_info, f, indent=2, ensure_ascii=False)

        # 驗證複製結果
        copied_files = os.listdir(deploy_dir)
        print(f"\n✅ 已複製 {len(copied_files)} 個檔案到 {deploy_dir}")
        print(f"📊 F1: {best_model['f1']:.4f}")

        # 列出複製的檔案
        print("\n📁 部署目錄內容:")
        for f in sorted(copied_files):
            size = os.path.getsize(os.path.join(deploy_dir, f))
            print(f"  - {f}: {size / 1024 / 1024:.1f} MB")
else:
    print("⚠️ 未達標或無可用模型")

## 1️⃣1️⃣ 推送到 GitHub (如果達標 ≥0.75)

In [ ]:
TARGET_F1 = 0.75

if 'best_model' in locals() and best_model['f1'] >= TARGET_F1:
    print(f"🎉 達標！F1 = {best_model['f1']:.4f} ≥ {TARGET_F1}")
    
    # 檢查模型檔案是否存在
    deploy_dir = "models/bullying_a100_best"
    model_files = [f for f in os.listdir(deploy_dir) 
                   if f.endswith(('.safetensors', '.bin', '.pt', '.pth'))]
    
    if not model_files:
        print("\n❌ 錯誤: 部署目錄中沒有模型權重檔案！")
        print("請檢查上一個 cell 的輸出，確認模型是否正確複製。")
    else:
        print(f"\n✅ 確認 {len(model_files)} 個模型檔案存在")
        for f in model_files:
            size = os.path.getsize(os.path.join(deploy_dir, f))
            print(f"  - {f}: {size / 1024 / 1024:.1f} MB")
        
        print("\n推送到 GitHub...\n")
        
        # Git LFS 設定
        print("📦 設定 Git LFS...")
        !git lfs install
        
        # 確保 .gitattributes 正確追蹤大檔案
        gitattributes_content = """# Model weights (large files)
models/**/*.safetensors filter=lfs diff=lfs merge=lfs -text
models/**/*.bin filter=lfs diff=lfs merge=lfs -text
models/**/*.pt filter=lfs diff=lfs merge=lfs -text
models/**/*.pth filter=lfs diff=lfs merge=lfs -text
models/**/*.ckpt filter=lfs diff=lfs merge=lfs -text

# Training data (large files)
data/**/*.json filter=lfs diff=lfs merge=lfs -text
data/**/*.csv filter=lfs diff=lfs merge=lfs -text
data/**/*.txt filter=lfs diff=lfs merge=lfs -text
"""
        
        with open('.gitattributes', 'w') as f:
            f.write(gitattributes_content)
        
        print("✅ .gitattributes 已更新")
        
        # 添加到 Git
        !git add .gitattributes
        !git add models/bullying_a100_best/
        
        # 檢查 Git 狀態
        print("\n📋 Git 狀態:")
        !git status --short
        
        # 檢查 LFS 追蹤狀態
        print("\n📦 Git LFS 追蹤的檔案:")
        !git lfs ls-files
        
        # 提交
        commit_msg = f"feat: A100 trained model (F1={best_model['f1']:.4f})"
        print(f"\n💾 提交訊息: {commit_msg}")
        !git commit -m "{commit_msg}"
        
        # 推送（包含 LFS 檔案）
        print("\n⬆️ 推送到 GitHub（包含 LFS 檔案）...")
        !git push origin main
        
        print("\n✅ 推送完成！")
        print(f"\n🎯 訓練結果:")
        print(f"  - 模型: {best_model['name']}")
        print(f"  - F1: {best_model['f1']:.4f}")
        print(f"  - 檔案: {', '.join(model_files)}")
        
elif 'best_model' in locals():
    print(f"⚠️ 未達標: F1 = {best_model['f1']:.4f} < {TARGET_F1}")
    print("模型未推送到 GitHub")
else:
    print("❌ 無可用模型")
    print("請檢查訓練過程是否有錯誤")

## 💾 備份到 Google Drive (可選)

In [ ]:
# 可選：備份模型到 Google Drive
# 如果 GitHub 推送失敗或想要額外備份，可執行此 cell

BACKUP_TO_DRIVE = False  # 設為 True 來啟用 Drive 備份

if BACKUP_TO_DRIVE and 'best_model' in locals():
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        
        # 設定備份路徑
        drive_backup_dir = f"/content/drive/MyDrive/CyberPuppy_Models/{best_model['name']}"
        os.makedirs(drive_backup_dir, exist_ok=True)
        
        # 複製模型到 Drive
        deploy_dir = "models/bullying_a100_best"
        print(f"📦 備份模型到 Google Drive...")
        print(f"目標: {drive_backup_dir}")
        
        for file in os.listdir(deploy_dir):
            src = os.path.join(deploy_dir, file)
            dst = os.path.join(drive_backup_dir, file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                size = os.path.getsize(dst) / 1024 / 1024
                print(f"  ✅ {file} ({size:.1f} MB)")
        
        print(f"\n✅ 備份完成！")
        print(f"📁 位置: {drive_backup_dir}")
        
    except Exception as e:
        print(f"❌ Drive 備份失敗: {e}")
        print("模型仍保存在 Colab 本地和 GitHub（如果推送成功）")
else:
    print("💡 提示: 如需備份到 Google Drive，請將 BACKUP_TO_DRIVE 設為 True")

## 1️⃣2️⃣ 訓練總結

In [ ]:
print("="*80)
print("🎯 A100 訓練總結")
print("="*80)

if 'results' in locals() and results:
    print("\n訓練結果:")
    for model in sorted(results, key=lambda x: x['f1'], reverse=True):
        status = "✅" if model['f1'] >= TARGET_F1 else "⚠️"
        print(f"  {status} {model['name']}: F1 = {model['f1']:.4f}")
    
    if 'best_model' in locals():
        print(f"\n🏆 最佳: {best_model['name']}")
        print(f"📊 F1: {best_model['f1']:.4f}")
        print(f"🎯 目標: {TARGET_F1}")
        
        if best_model['f1'] >= TARGET_F1:
            print("\n✅ 成功！模型已推送到 GitHub")
        else:
            gap = TARGET_F1 - best_model['f1']
            print(f"\n⚠️ 距離目標: {gap:.4f}")
else:
    print("\n❌ 無訓練結果")

print("\n" + "="*80)